In [1]:
%run 'C:\dev\prod\metrikaHelper.py'
import pandas as pd
import psycopg2

In [2]:
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}

In [3]:
allDomainsStats = []

In [10]:
project = u'doorlock.ru'
VipProjectInfo = getVipProjectByDomain(project)
print VipProjectInfo

[{'metrikalogin': '', 'projectid': 2236050L, 'domain': 'doorlock.ru', 'metrikapassword': ''}, {'metrikalogin': 'doorlockru', 'projectid': 2236303L, 'domain': 'doorlock.ru', 'metrikapassword': 'Asdqwe9'}, {'metrikalogin': '', 'projectid': 2236707L, 'domain': 'doorlock.ru', 'metrikapassword': ''}, {'metrikalogin': 'doorlockru', 'projectid': 2363184L, 'domain': 'doorlock.ru', 'metrikapassword': 'Asdqwe9'}]


In [15]:
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[1]['metrikalogin'],'yandexPassword':VipProjectInfo[1]['metrikapassword'],'domain':VipProjectInfo[1]['domain']})
print metrikaCredentials

{'Token': u'234c2324fb7b4139ba79c2b43be01076', 'counterID': [36936]}


Получаем статистику Яндекс.Метрики по пользователям

In [16]:
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[1]['projectid'])])
otherMetrics = 'ym:s:visits'
metrics = ','.join([otherMetrics,goalsMetrics])
dimensions='ym:s:userIDHash,ym:s:dateTime,ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMContent,ym:s:UTMTerm'
date1 = '2016-01-01'
date2 = '2016-03-30'
token = metrikaCredentials['Token']
counterId = metrikaCredentials['counterID'][0]
params = '&'.join(['dimensions={0}'.format(dimensions),
                   'metrics={0}'.format(metrics),
                   'date1={0}'.format(date1),
                  'date2={0}'.format(date2),
                  'ids={0}'.format(counterId),
                  'oauth_token={0}'.format(token),
                  'limit=100000',
                  'offset=1',
                  "filters=ym:s:UTMSource=='cubo'"])
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
print url
df = pd.read_csv(url,encoding='utf8')
print len(df)
if len(df) < 500:
    print 'Not enough data'
else:
    df.rename(columns={u'﻿"Идентификатор посетителя (uid)"': 'userID',
                        u'Дата и время визита': 'datetime',
                        u'Визиты': 'visits'
                       }, inplace=True)
    for i in xrange(0,len(df)):
        df.loc[i,'sumCPAVisits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df['domain'] = VipProjectInfo[0]['domain']
    df = df[~df[u'userID'].isin([u'Итого и средние'])]
    cpaVisitsCount = df['sumCPAVisits'].sum()
    print cpaVisitsCount
    df = df[~df[u'UTM Content'].isin([u'{PTYPE}.{POS}.{BM}',
                                      u'{keyword}.{position_type}.{position}.{addphrases}',
                                      '{PHRASE}.{PTYPE}.{POS}.{BM}',
                                      '{position_type}.{position}.{addphrases}'])]

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:userIDHash,ym:s:dateTime,ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMContent,ym:s:UTMTerm&metrics=ym:s:visits,ym:s:goal2309269visits&date1=2016-01-01&date2=2016-03-30&ids=36936&oauth_token=234c2324fb7b4139ba79c2b43be01076&limit=100000&offset=1&filters=ym:s:UTMSource=='cubo'
62277
56.0


In [11]:
df.to_excel('D:\\mk.xlsx')

In [17]:
# Оставляем в статистике только пользователей с достижениями целей > 0
userStat = df[['userID','sumCPAVisits']].groupby([u'userID'],as_index=False).sum()
df2 = df[df[u'userID'].isin(userStat[userStat['sumCPAVisits'] > 0]['userID'])]
usersWithConversionsCount = len(df2['userID'].unique())
# Оставляем в статистике только пользователей с достижениями целей > 0 и количеством посещений > 1 
usersWithMore1visits = (df2['userID'].value_counts() > 1)
df3 = df[df[u'userID'].isin(usersWithMore1visits[usersWithMore1visits == True].index)]
# Если в статистике осталось > 0 записей, то
if len(df3) > 0:
    final_df = df3[['userID','datetime','UTM Source','UTM Content','UTM Term','visits','sumCPAVisits','domain']]
#     Получаем статистику пользователей, у которых есть посещения с разными utm-метками
    needUsers = (pd.DataFrame([list(item) for item in final_df.groupby(['userID','UTM Source','UTM Content','UTM Term']).groups.keys()])[0].value_counts() > 1)
    df_need = final_df[final_df['userID'].isin(needUsers[needUsers].index)]
    if len(df_need) > 0:
        itogo_df = pd.DataFrame()
        # Для каждого пользователя получаем его статистику и убираем с конца посещения, у которых 0 достижений целей
        for user in df_need['userID'].unique():
            small_df = df_need[df_need['userID'] == user]
            t = True
            while t:
                small_df.reset_index(drop=True)
                last_row = small_df.tail(1)
                if last_row['sumCPAVisits'].values[0] > 0.0:
                    t = False
                else:
                    small_df.drop(small_df.tail(1)['sumCPAVisits'].index,inplace=True)
            itogo_df = pd.concat([itogo_df,small_df])
        usersWithMore1visits = (itogo_df['userID'].value_counts() > 1)
        itogo_df = itogo_df[itogo_df[u'userID'].isin(usersWithMore1visits[usersWithMore1visits == True].index)]
        if len(itogo_df) == 0:
            print 'No associated conversions'
    else:
        print 'No associated conversions'
else:
    print 'No users with conversions and more than 1 visits'

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
itogo_df

,userID,datetime,UTM Source,UTM Content,UTM Term,visits,sumCPAVisits,domain
2009,147434581,2016-01-31 15:54:04,cubo,none.0.no,механизм для раздвижных межкомнатных дверей,1,1,doorlock.ru
2010,147434581,2016-02-01 18:31:00,cubo,Не определено,Не определено,1,1,doorlock.ru
2332,174816717,2016-03-05 15:37:12,cubo,none.0.no,трос для велосипеда,1,0,doorlock.ru
2333,174816717,2016-03-06 22:39:23,cubo,none.0.no,противоугонный трос для велосипеда,1,1,doorlock.ru
15953,1112430346,2016-03-20 18:44:25,cubo,none.0.no,замок для велосипеда цена,1,1,doorlock.ru
15954,1112430346,2016-03-20 19:04:19,cubo,none.0.no,велосипедный замок,1,1,doorlock.ru
17330,1199481224,2016-03-14 21:29:16,cubo,none.0.no,велозамок купить,1,0,doorlock.ru
17331,1199481224,2016-03-15 08:56:59,cubo,none.0.no,велозамок цепь,1,0,doorlock.ru
17332,1199481224,2016-03-24 18:15:52,cubo,none.0.no,замок велосипедный противоугонный,1,0,doorlock.ru
17333,1199481224,2016-03-27 07:43:37,cubo,none.0.no,велосипедная цепь с замком,1,1,doorlock.ru


In [17]:
itogo_df.to_clipboard(excel=True,encoding='cp1251')

In [19]:
domainStatistics = {'Domain':VipProjectInfo[0]['domain'],
                    'All conversions':cpaVisitsCount,
                    'All users with conversions':usersWithConversionsCount,
                   'Unique users with associated conversions':len(itogo_df['userID'].unique()),
                   'Conversions of associated conversions users':itogo_df['sumCPAVisits'].sum(),
                   'Percent of associated conversions':itogo_df['sumCPAVisits'].sum()/cpaVisitsCount}
allDomainsStats.append(domainStatistics)
pd.DataFrame(allDomainsStats).to_clipboard(excel=True,encoding='cp1251')